This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/wallaroo-features/wallaroo-model-endpoints).

## Internal Pipeline Inference URL Tutorial

Wallaroo provides the ability to perform inferences through deployed pipelines via both internal and external inference URLs.  These inference URLs allow inferences to be performed by submitting data to the internal or external URL with the inference results returned in the same format as the [InferenceResult Object](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/#run-inference-through-a-pipeline).

**Internal URLs** are available only through the internal Kubernetes environment hosting the Wallaroo instance as demonstrated in this tutorial.
**External URLs** are available outside of the Kubernetes environment, such as the public internet.  These are demonstrated in the External Pipeline Deployment URL Tutorial.

The following tutorial shows how to set up an environment and demonstrates how to use the Internal Deployment URL.  This example provides the following:

* `alohacnnlstm.zip`:  Aloha model used as part of the [Aloha Quick Tutorial](https://docs.wallaroo.ai/wallaroo-tutorials/wallaroo-quick-start-aloha/).
* For Arrow enabled instances:
  * `data_1.df.json`, `data_1k.df.json` and `data_25k.df.json`:  Sample data used for testing inferences with the sample model.
* For Arrow distabled instances:
  * `data_1.json`, `data_1k.json` and `data_25k.json`:  Sample data used for testing inferences with the sample model.

For our example, we will perform the following:

* Create a workspace for our work.
* Upload the Aloha model.
* Create a pipeline that can ingest our submitted data, submit it to the model, and export the results.
* Run a sample inference through our pipeline via the SDK to demonstrate the inference is accurate.
* Run a sample inference through our pipeline's Internal URL and store the results in a file.

All sample data and models are available through the [Wallaroo Quick Start Guide Samples repository](https://github.com/WallarooLabs/quickstartguide_samples).

## Open a Connection to Wallaroo

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

In [41]:
import wallaroo
from wallaroo.object import EntityNotFoundError
import pandas as pd

# used to display dataframe information without truncating
from IPython.display import display
pd.set_option('display.max_colwidth', None)

In [42]:
# Client connection from local Wallaroo instance

# wl = wallaroo.Client()

# SSO login through keycloak

# wallarooPrefix = "YOUR PREFIX"
# wallarooSuffix = "YOUR SUFFIX"

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
                    auth_type="sso")

### Arrow Support

As of the 2023.1 release, Wallaroo provides support for dataframe and Arrow for inference inputs.  This tutorial allows users to adjust their experience based on whether they have enabled Arrow support in their Wallaroo instance or not.

If Arrow support has been enabled, `arrowEnabled=True`. If disabled or you're not sure, set it to `arrowEnabled=False`

The examples below will be shown in an arrow enabled environment.

In [53]:
import os
arrowEnabled=True
os.environ["ARROW_ENABLED"]=f"{arrowEnabled}"


## Create the Workspace

We will create a workspace to work in and call it the `urldemoworkspace`, then set it as current workspace environment.  We'll also create our pipeline in advance as `urldemopipeline`.

The model to be uploaded and used for inference will be labeled as `urldemomodel`.  Modify these to your organizations requirements.

Once complete, the workspace will be created or, if already existing, set to the current workspace to host the pipelines and models.

In [44]:
workspace_name = 'urldemoworkspace'
pipeline_name = 'urldemopipeline'
model_name = 'urldemomodel'
model_file_name = './alohacnnlstm.zip'

In [45]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(pipeline_name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(pipeline_name)
    return pipeline

In [46]:
workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

pipeline = get_pipeline(pipeline_name)
pipeline

name,urldemopipeline
created,2023-02-16 23:09:39.356010+00:00
last_updated,2023-02-16 23:09:39.356010+00:00
deployed,(none)
tags,
versions,1bafb16d-313b-4951-9ae0-d41355dc8e6d
steps,


We can verify the workspace is created the current default workspace with the `get_current_workspace()` command.

In [47]:
wl.get_current_workspace()

{'name': 'urldemoworkspace', 'id': 8, 'archived': False, 'created_by': 'f57ceacc-0d73-4212-bba3-0c8ee7ac3042', 'created_at': '2023-02-16T23:09:37.777334+00:00', 'models': [], 'pipelines': [{'name': 'urldemopipeline', 'create_time': datetime.datetime(2023, 2, 16, 23, 9, 39, 356010, tzinfo=tzutc()), 'definition': '[]'}]}

# Upload the Models

Now we will upload our models.  Note that for this example we are applying the model from a .ZIP file.  The Aloha model is a [protobuf](https://developers.google.com/protocol-buffers) file that has been defined for evaluating web pages, and we will configure it to use data in the `tensorflow` format.

In [48]:
model = wl.upload_model(model_name, model_file_name).configure("tensorflow")

## Deploy The Pipeline
Now that we have a model that we want to use we will create a deployment for it. 

We will tell the deployment we are using a tensorflow model and give the deployment name and the configuration we want for the deployment.

In [49]:
pipeline.add_model_step(model)
pipeline.deploy()

name,urldemopipeline
created,2023-02-16 23:09:39.356010+00:00
last_updated,2023-02-16 23:09:44.118173+00:00
deployed,True
tags,
versions,"0b2ad283-4c73-4fbb-9890-c79d098764c4, 1bafb16d-313b-4951-9ae0-d41355dc8e6d"
steps,urldemomodel


We can verify that the pipeline is running and list what models are associated with it.

In [50]:
pipeline.status()

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.244.13.9',
   'name': 'engine-7b9554f474-4g8n7',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'urldemopipeline',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'urldemomodel',
      'version': 'cfba8b33-6e6b-41e7-bb83-fe322518b51a',
      'sha': 'd71d9ffc61aaac58c2b1ed70a2db13d1416fb9d3f5b891e5e4e2e97180fe22f8',
      'status': 'Running'}]}}],
 'engine_lbs': [{'ip': '10.244.17.16',
   'name': 'engine-lb-ddd995646-d55fx',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': []}

## Interferences

### Infer 1 row

Now that the pipeline is deployed and our Aloha model is in place, we'll perform a smoke test to verify the pipeline is up and running properly.  We'll use the `infer_from_file` command to load a single encoded URL into the inference engine and print the results back out.

The result should tell us that the tokenized URL is legitimate (0) or fraud (1).  This sample data should return close to 0.

In [54]:
if arrowEnabled is True:
    result = pipeline.infer_from_file('./data/data_1.df.json')
else:
    result = pipeline.infer_from_file("./data/data_1.json")
display(result)

,time,in.text_input,out.gozi,out.main,out.banjori,out.corebot,out.kraken,out.matsnu,out.cryptolocker,out.qakbot,out.locky,out.simda,out.ramdo,out.suppobox,out.ramnit,out.dircrypt,out.pykspa,check_failures
0,2023-02-16 23:11:27.718,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 28, 16, 32, 23, 29, 32, 30, 19, 26, 17]",[2.028935e-05],[0.997564],[0.0015195842],[0.98291475],[0.00031977228],[0.010341614],[0.012099549],[0.016155055],[0.011029261],[1.7933368e-26],[0.0062362333],[1.3889844e-27],[0.0009985747],[4.759116e-05],[0.008038961],0


### Batch Inference

Now that our smoke test is successful, we will retrieve the Internal Deployment URL and perform an inference by submitting our data through a `curl` command as detailed below.

* **IMPORTANT NOTE**:  The `_deployment._url()` method will return an **internal** URL when using Python commands from within the Wallaroo instance - for example, the Wallaroo JupyterHub service.  When connecting via an external connection, `_deployment._url()` returns an **external** URL.  External URL connections requires [the authentication be included in the HTTP request](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-api-guide/), and that [Model Endpoints Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-model-endpoints-guide/) external endpoints are enabled in the Wallaroo configuration options.

In [55]:
inference_url = pipeline._deployment._url()
print(inference_url)
connection =wl.mlops().__dict__
token = connection['token']
print(token)

https://doc-test.api.wallaroocommunity.ninja/v1/api/pipelines/infer/urldemopipeline-4
eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICI2dnlWSU1JSDE3a09HWUZGQW5DRVVMOVFzOFFWZ0NSSTZJbmVJdGpFVnBrIn0.eyJleHAiOjE2NzY1ODkxNDcsImlhdCI6MTY3NjU4OTA4NywiYXV0aF90aW1lIjoxNjc2NTc4NDEwLCJqdGkiOiIxMDQ2MGJmMS03ODlmLTQ2NDEtOGYwMS01NGY4ZWMyZWY2NDgiLCJpc3MiOiJodHRwczovL2RvYy10ZXN0LmtleWNsb2FrLndhbGxhcm9vY29tbXVuaXR5Lm5pbmphL2F1dGgvcmVhbG1zL21hc3RlciIsImF1ZCI6WyJtYXN0ZXItcmVhbG0iLCJhY2NvdW50Il0sInN1YiI6ImY1N2NlYWNjLTBkNzMtNDIxMi1iYmEzLTBjOGVlN2FjMzA0MiIsInR5cCI6IkJlYXJlciIsImF6cCI6InNkay1jbGllbnQiLCJzZXNzaW9uX3N0YXRlIjoiNjFmNmU2NjQtOWRiYi00ZjQxLTkyYjMtOGQ2MGU1NzdkMDg5IiwiYWNyIjoiMCIsInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJkZWZhdWx0LXJvbGVzLW1hc3RlciIsIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iXX0sInJlc291cmNlX2FjY2VzcyI6eyJtYXN0ZXItcmVhbG0iOnsicm9sZXMiOlsibWFuYWdlLXVzZXJzIiwidmlldy11c2VycyIsInF1ZXJ5LWdyb3VwcyIsInF1ZXJ5LXVzZXJzIl19LCJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50IiwibWFuYWdlLWFjY291bnQtbG

In [56]:
if arrowEnabled is True:
    dataFile="./data/data_25k.df.json"
    contentType="application/json; format=pandas-records"
else:
    dataFile="./data/data_25k.json"
    contentType="application/json"

In [57]:
!curl -X POST {inference_url} -H "Authorization: Bearer {token}" -H "Content-Type:{contentType}" --data @{dataFile} > curl_response.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 34.3M  100 16.3M  100 18.0M  1131k  1248k  0:00:15  0:00:14  0:00:01 3697k  0:00:14  0:00:14 --:--:-- 4536k


## Undeploy Pipeline

When finished with our tests, we will undeploy the pipeline so we have the Kubernetes resources back for other tasks.

**IMPORTANT NOTE**:  For the External Pipeline Deployment URL Tutorial, this pipeline will have to be deployed to make the External Deployment URL available.

In [58]:
pipeline.undeploy()

name,urldemopipeline
created,2023-02-16 23:09:39.356010+00:00
last_updated,2023-02-16 23:09:44.118173+00:00
deployed,False
tags,
versions,"0b2ad283-4c73-4fbb-9890-c79d098764c4, 1bafb16d-313b-4951-9ae0-d41355dc8e6d"
steps,urldemomodel
